# Experiment using model 1

Description


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
from everywhere.datasets.load_data import get_users_data, get_events_data, get_train_data, get_user_friends_data, get_event_attendees_data

Numpy version: 1.24.2
Pandas version: 1.5.3
Everywhere version: 0.0.0


In [ ]:
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Everywhere version: {everywhere.__version__}")

## Model theoretical explanation

This model is based in ...


In [3]:
# Parameters
batch_size = 32
n_layers = 5
#epochs = 10
EPOCHS = 10

## Data prep


In [4]:
data_path = "/Users/diegosanmartin/Desktop/Diego/Dsanmart/everywhere-reco/examples/local_data/"
"""df_events = pd.read_csv(data_path + "events.csv")
df_users = pd.read_csv(data_path + "users.csv")
df_user_friends = pd.read_csv(data_path + "user_friends.csv")
df_event_attendees = pd.read_csv(data_path + "event_attendees.csv")
df_train = pd.read_csv(data_path + "train.csv")"""

'df_events = pd.read_csv(data_path + "events.csv")\ndf_users = pd.read_csv(data_path + "users.csv")\ndf_user_friends = pd.read_csv(data_path + "user_friends.csv")\ndf_event_attendees = pd.read_csv(data_path + "event_attendees.csv")\ndf_train = pd.read_csv(data_path + "train.csv")'

In [7]:
from dotenv import load_dotenv
load_dotenv()
get_train_data(data_path + "train.csv")

Local dataset loaded :)


,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0
...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0


In [8]:
df_events.head()

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


In [13]:
print(df_event_attendees.shape)
df_event_attendees.head()

(24144, 5)


,event,yes,maybe,invited,no
0,1159822043,1975964455 252302513 4226086795 3805886383 142...,2733420590 517546982 1350834692 532087573 5831...,1723091036 3795873583 4109144917 3560622906 31...,3575574655 1077296663
1,686467261,2394228942 2686116898 1056558062 3792942231 41...,1498184352 645689144 3770076778 331335845 4239...,1788073374 733302094 1830571649 676508092 7081...,NaN
2,1186208412,NaN,3320380166 3810793697,1379121209 440668682,1728988561 2950720854
3,2621578336,NaN,NaN,NaN,NaN
4,855842686,2406118796 3550897984 294255260 1125817077 109...,2671721559 1761448345 2356975806 2666669465 10...,1518670705 880919237 2326414227 2673818347 332...,3500235232


In [10]:
df_users.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [11]:
df_user_friends.head()

,user,friends
0,3197468391,1346449342 3873244116 4226080662 1222907620 54...
1,3537982273,1491560444 395798035 2036380346 899375619 3534...
2,823183725,1484954627 1950387873 1652977611 4185960823 42...
3,1872223848,83361640 723814682 557944478 1724049724 253059...
4,3429017717,4253303705 2130310957 1838389374 3928735761 71...


In [31]:
def get_friends_status(row, event_attendees_df):
    """Returns the number of friends attending, not attending, maybe attending, and invited to the event."""

    event = row['event']
    friends = str(row['friends']).split(' ')
    ppl_attending = str(event_attendees_df.iloc[np.where(event_attendees_df['event']==event)[0][0]]['yes']).split(' ')
    ppl_not_attending = str(event_attendees_df.iloc[np.where(event_attendees_df['event']==event)[0][0]]['no']).split(' ')
    ppl_maybe_attending = str(event_attendees_df.iloc[np.where(event_attendees_df['event']==event)[0][0]]['maybe']).split(' ')
    ppl_invited = str(event_attendees_df.iloc[np.where(event_attendees_df['event']==event)[0][0]]['invited']).split(' ')
    friends_attending = list(set(friends).intersection(set(ppl_attending)))
    friends_not_attending = list(set(friends).intersection(set(ppl_not_attending)))
    friends_maybe_attending = list(set(friends).intersection(set(ppl_maybe_attending)))
    friends_invited = list(set(friends).intersection(set(ppl_invited)))
    
    return [len(friends_attending), len(friends_not_attending), len(friends_maybe_attending), len(friends_invited)]

def count_words(row, col_name):
    if(type(row[col_name]) != str):
        return 0
    return len(row[col_name].split())

def get_friends_attendee_nums(train_df, friends_df, event_attendees_df):
    # merge train_df with friends_df
    merged_df = pd.merge(train_df, friends_df, how='inner', left_on='user', right_on='user')
    merged_df['friends_attending'], merged_df['friends_not_attending'], merged_df['friends_maybe_attending'], merged_df['friends_invited'] = zip(*merged_df.apply(lambda row: get_friends_status(row, event_attendees_df), axis=1))
    # convert friends column to number of friends
    merged_df['friends'] = merged_df.apply (lambda row: count_words(row, 'friends'), axis=1)
    return merged_df

In [32]:
df_train = get_friends_attendee_nums(df_train, df_user_friends, df_event_attendees)
df_train

,user,event,invited,timestamp,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,3,0,3
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,1
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0,862,0,0,0,2
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0,862,1,0,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,0,1,0
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,0,0,1
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,1,1,3
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,1,1,0


In [33]:
def get_event_attendee_nums(train_df, event_attendees_df):
    merged_df = pd.merge(train_df, event_attendees_df, how='inner', left_on='event', right_on='event')
    merged_df['users_yes'] = merged_df.apply (lambda row: count_words(row, 'yes'), axis=1)
    merged_df['users_no'] = merged_df.apply (lambda row: count_words(row, 'no'), axis=1)
    merged_df['users_maybe'] = merged_df.apply (lambda row: count_words(row, 'maybe'), axis=1)
    merged_df['users_invited_count'] = merged_df.apply (lambda row: count_words(row, 'invited_y'), axis=1)

    return merged_df

df_train = get_event_attendee_nums(df_train, df_event_attendees)
df_train

,user,event,invited_x,timestamp,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,friends_invited,yes,maybe,invited_y,no,invited_count
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,3,0,3,8,2,243449741 2705495661 2053207729 2228554219 229...,25,23
1,1906220044,1918771225,0,2012-10-01 05:08:48.533000+00:00,0,0,930,1,0,0,0,8,2,243449741 2705495661 2053207729 2228554219 229...,25,23
2,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,1,10,6,2823348035 1461073321 1603391812 1511769431 35...,2,122
3,1302145719,1502284248,0,2012-09-30 13:31:33.569000+00:00,0,0,2265,0,0,0,6,10,6,2823348035 1461073321 1603391812 1511769431 35...,2,122
4,3194014105,1502284248,0,2012-10-01 17:20:14.536000+00:00,0,0,619,0,0,0,0,10,6,2823348035 1461073321 1603391812 1511769431 35...,2,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,0,1,0,111,81,4089136524 3084736394 4069540035 3458310857 31...,26,477
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,0,0,1,37,37,3641702874 13080990 4118624064 1764266106 1354...,2,313
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,1,1,3,181,66,685339129 3485457810 1790249421 476313752 4087...,14,696
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,1,1,0,436,199,4175140272 1803249223 2535143415 1981386687 67...,1340,842


In [38]:
df_events.columns

Index(['event_id', 'user_id', 'start_time', 'city', 'state', 'zip', 'country',
       'lat', 'lng', 'c_1',
       ...
       'c_92', 'c_93', 'c_94', 'c_95', 'c_96', 'c_97', 'c_98', 'c_99', 'c_100',
       'c_other'],
      dtype='object', length=110)

In [41]:
df_events_clean = df_events.drop(columns=['user_id', 'start_time', 'city', 'state', 'zip', 'country', 'lat', 'lng'])
# Merge with events
df_train = pd.merge(df_train, df_events_clean, how='inner', left_on='event', right_on='event_id')
df_train

,user,event,invited_x,timestamp,interested,not_interested,friends,friends_attending,friends_not_attending,friends_maybe_attending,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,3,0,...,0,0,0,0,0,0,0,0,0,2
1,1906220044,1918771225,0,2012-10-01 05:08:48.533000+00:00,0,0,930,1,0,0,...,0,0,0,0,0,0,0,0,0,2
2,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,862,0,0,0,...,0,0,0,0,0,0,0,0,0,24
3,1302145719,1502284248,0,2012-09-30 13:31:33.569000+00:00,0,0,2265,0,0,0,...,0,0,0,0,0,0,0,0,0,24
4,3194014105,1502284248,0,2012-10-01 17:20:14.536000+00:00,0,0,619,0,0,0,...,0,0,0,0,0,0,0,0,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15393,4293103086,2750873665,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,0,1,...,0,0,0,0,0,0,0,0,0,80
15394,4293103086,4084655790,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,0,0,...,0,0,0,0,0,0,0,0,0,80
15395,4293103086,598708806,0,2012-12-08 03:59:43.169000+00:00,0,0,480,1,1,1,...,0,0,0,0,0,0,0,0,0,60
15396,4293103086,604179853,0,2012-12-08 03:59:43.169000+00:00,0,0,480,0,1,1,...,0,0,0,0,0,0,0,0,0,24


In [42]:
df_train = df_train.drop(columns=['invited_y', 'timestamp'])

In [45]:
df_train["interested"].value_counts()

0    11267
1     4131
Name: interested, dtype: int64

## Model training


In [47]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler
from collections import Counter

data=df_train.copy()
X = data.drop('Interested', axis = 1)
y = data['Interested']
print('Original dataset shape {}'.format(Counter(y)))
cc = ClusterCentroids(random_state=42)
X_res, y_res = cc.fit_resample(X, y)
print('Resampled dataset shape {}'.format(Counter(y_res)))

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
# preds = model.test

## Evaluation


In [ ]:
# model_auc = auc(true, preds)

## Summary
